To switch to dbt profile with the `arn:aws:iam::026090528544:role/GlueInteractiveSessionClientRole-dbt-CLI-demo` role that I created

In [ ]:
nano ~/.aws/config

# In nano add this profile
[profile dbt-glue]
role_arn = arn:aws:iam::026090528544:role/GlueInteractiveSessionClientRole-dbt-CLI-demo
source_profile = Joshua_Harris_ETL
region = ap-southeast-2

# switch profile for the terminal session
export AWS_PROFILE=dbt-glue

# check your cureent profile
aws sts get-caller-identity

lets make a csv file with example data

In [92]:
# pip install pandas numpy

In [93]:
from pandas import DataFrame
from numpy import nan

import numpy as np

def permute_dataframe(df, random_state=None):
    """
    Returns a new DataFrame with the rows randomly permuted.
    Each call produces a different order unless random_state is set.
    """
    return df.sample(frac=1, random_state=random_state).reset_index(drop=True)


dirty_df = DataFrame({
    'patient_id': ["A001", "A002", "A003", "A004", "A005", ""], 
    'sex': ["M", "F", "0", "1", "Male", "fem"],
    'medication': [None, '', None, nan, None, 'statin'],
    'age': [nan, 32, 214, 21, 0, ""]
    })


# Example usage of permute_dataframe:

import random
permuted_df = permute_dataframe(dirty_df, random_state=random.randint(0, 1_000_000))


with open('../data/patient_info.csv', 'w') as f:
    permuted_df.to_csv(f, index=False)

lets upload this file to s3

In [94]:
%%bash
DATETIME=$(date +%Y%m%d%H%M%S)
DATA_NAME="patient_info"
aws s3 cp "../data/patient_info.csv" "s3://acdc-dbt-test-raw/data/${DATA_NAME}/date_time=${DATETIME}/${DATA_NAME}.csv" --profile "default"

upload: ../data/patient_info.csv to s3://acdc-dbt-test-raw/data/patient_info/date_time=20250725135117/patient_info.csv


lets make some other data types

In [95]:
from pandas import DataFrame
from numpy import nan

dirty_df = DataFrame({
    'sample_id': ["SAM0001", "SAM0002", "SAM0003", "SAM0004", "SAM0005", ""],
    'patient_id': ["A001", "A002", "A003", "A004", "A005", ""], 
    'storage_medium': ["nitrogen", "air", "nitrogen", "air", "nitrogen", "air"],
    'storage_date': ["2022-01-01", "2022-01-02", "2022-01-03", "2022-01-04", "2022-01-05", "2022-01-06"],
    'volume_ul': [0, 486, 4, 4000, 5000, 6000]
    })

permuted_df = permute_dataframe(dirty_df, random_state=random.randint(0, 1_000_000))

with open('../data/sample.csv', 'w') as f:
    permuted_df.to_csv(f, index=False)

In [96]:
%%bash
DATETIME=$(date +%Y%m%d%H%M%S)
DATA_NAME="sample"
aws s3 cp "../data/sample.csv" "s3://acdc-dbt-test-raw/data/${DATA_NAME}/date_time=${DATETIME}/${DATA_NAME}.csv" --profile "default"

upload: ../data/sample.csv to s3://acdc-dbt-test-raw/data/sample/date_time=20250725135118/sample.csv
